In [1]:

!pip install --upgrade transformers sentencepiece datasets sacrebleu rouge-score accelerate --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 23.4 MB/s eta 0:00:00


In [2]:
import sys
import pandas as pd
import torch
from datasets import Dataset, DatasetDict

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

print("Python:", sys.version.splitlines()[0])
print("Transformers version:", transformers.__version__)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Transformers version: 4.57.1


In [4]:
train_df = pd.read_csv("/content/train_en.csv")
val_df   = pd.read_csv("/content/val_en.csv")
test_df  = pd.read_csv("/content/test_en.csv")

print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)

# Autodetect columns (first two columns expected: input, response)
input_col  = train_df.columns[0]
target_col = train_df.columns[1]
print("Input column:", input_col)
print("Target column:", target_col)


Train: (1140, 8)
Val: (229, 8)
Test: (153, 8)
Input column: id
Target column: source


In [5]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

# Quick peek
print(dataset["train"][0])


{'id': 'dlg_cbb0155f815386275f372989ec609d9f', 'source': 'train.csv', 'type': 'dialogue', 'prompt': 'Is this something I should be worried about? Should I do something about it?', 'response': "The answer depends on how the other areas of your son's life are doing.Is he happy or does he seem happy, playing alone during recess?Does he have friends in other social circles besides the students whom he's with at recess?How is his academic progress?How is his social integration among his classmates?Are there any special or unusual circumstances in the home and family environment?Go through this list to form a fuller idea of whether your son simply likes alone time and takes this option during recess, or if any if the above areas show stress or difficulty for him and which need to be further understood and handled.", 'lang': 'en', 'orig_id': None, 'fp': 'cbb0155f815386275f372989ec609d9f'}


In [6]:
model_name = "google/mt5-small"   # good balance of size & multilingual ability

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Loaded model & tokenizer:", model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded model & tokenizer: google/mt5-small


In [7]:
MAX_LEN = 128

def preprocess(batch):
    # batch is a dict of lists
    inputs = tokenizer(batch[input_col], truncation=True, padding="max_length", max_length=MAX_LEN)
    labels = tokenizer(batch[target_col], truncation=True, padding="max_length", max_length=MAX_LEN)

    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_ds = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
print("Tokenized datasets:", {k: len(v) for k, v in tokenized_ds.items()})


Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Tokenized datasets: {'train': 1140, 'validation': 229, 'test': 153}


In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training size info (used to compute fallback eval/save steps)
train_size = len(tokenized_ds["train"])
batch_size = 4   # you can change this according to your GPU
num_epochs = 6   # reduce/increase based on your compute budget
print("train_size:", train_size, "batch_size:", batch_size, "num_epochs:", num_epochs)


train_size: 1140 batch_size: 4 num_epochs: 6


In [9]:
# Define batch_size and num_epochs
batch_size = 4    # You can increase it based on your GPU's memory capacity
num_epochs = 6    # You can adjust the number of epochs for training

# Define the output directory where model checkpoints will be saved
output_dir = "./te_chatbot_model"  # You can modify this path to your preference

# Simplified TrainingArguments for transformers version 4.57.1
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,          # Save model every 500 steps
    eval_steps=500,          # Evaluate every 500 steps
    save_total_limit=2       # Keep 2 checkpoints max
)

print("TrainingArguments set successfully for transformers v4.57.1")


TrainingArguments set successfully for transformers v4.57.1


In [10]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained model (mT5-small or any other suitable model)
model_name = "google/mt5-small"  # Replace with your model name if different
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Model loaded successfully.")


Model loaded successfully.


In [11]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your CSV files into pandas DataFrames
train_df = pd.read_csv("/content/train_en.csv")
val_df = pd.read_csv("/content/val_en.csv")
test_df = pd.read_csv("/content/test_en.csv")

# Check the shape of the data
print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

# Convert pandas DataFrames to HuggingFace datasets
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

# Check if dataset is loaded correctly
print(dataset)


Train shape: (1140, 8)
Validation shape: (229, 8)
Test shape: (153, 8)
DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 1140
    })
    validation: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 229
    })
    test: Dataset({
        features: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp'],
        num_rows: 153
    })
})


In [12]:
from transformers import AutoTokenizer

# Load the tokenizer for mT5 (or your model)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [13]:
def preprocess(example):
    # Tokenize 'prompt' for the input text and 'response' for the target text
    inputs = tokenizer(example["prompt"], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(example["response"], truncation=True, padding='max_length', max_length=128)

    # Store the tokenized labels in 'labels'
    inputs["labels"] = labels["input_ids"]
    return inputs


In [14]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")  # Or your model

# Tokenize the dataset using the new column names ('prompt' and 'response')
def preprocess(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(example["response"], truncation=True, padding='max_length', max_length=128)

    inputs["labels"] = labels["input_ids"]
    return inputs

# Tokenize the dataset
tokenized_ds = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# Verify tokenized dataset (checking a sample)
print(tokenized_ds["train"][0])  # Verify tokenization


Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

{'input_ids': [2485, 714, 9066, 336, 3609, 390, 3301, 135138, 1388, 291, 259, 65717, 336, 342, 9066, 1388, 609, 291, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [486, 20954, 29564, 263, 351, 2606, 287, 1904, 259, 44326, 304, 772, 1065, 277, 263, 4256, 418, 342, 347, 260, 15206, 790, 14280, 631, 259

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
print("Model loaded successfully!")


Model loaded successfully!


In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [17]:
# Define batch_size and num_epochs
batch_size = 4
num_epochs = 6

output_dir = "./en_chatbot_model"

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2
)

print("TrainingArguments ready!")


TrainingArguments ready!


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

print("Trainer initialized successfully!")


/tmp/ipython-input-2274696399.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer initialized successfully!


In [19]:

import os, sys, types

# Ensure Trainer won't try to use WANDB
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"

# Create a minimal fake wandb module that has the functions Trainer expects
fake_wandb = types.SimpleNamespace(
    init=lambda *a, **k: None,
    log=lambda *a, **k: None,
    finish=lambda *a, **k: None,
    run=types.SimpleNamespace(name="disabled"),
    config={},
)

# Inject it into sys.modules so imports will find it
sys.modules["wandb"] = fake_wandb

# Now remove the WandbCallback from the trainer (if it was added)
try:
    from transformers.integrations import WandbCallback
    trainer.remove_callback(WandbCallback)
except Exception:
    # ignore if not present or removal fails
    pass

print("WandB monkey-patched and WandbCallback removed (if present). Now run trainer.train() again.")


WandB monkey-patched and WandbCallback removed (if present). Now run trainer.train() again.


In [20]:
trainer.train()


Step,Training Loss
50,15.794200
100,7.273400
150,4.566200
200,4.102700
250,3.706400
300,3.668400
350,3.413000
400,3.539600
450,3.407600
500,3.493300


TrainOutput(global_step=1710, training_loss=3.7677649012783117, metrics={'train_runtime': 729.3261, 'train_samples_per_second': 9.379, 'train_steps_per_second': 2.345, 'total_flos': 904161971404800.0, 'train_loss': 3.7677649012783117, 'epoch': 6.0})

In [21]:
metrics = trainer.evaluate(tokenized_ds["test"])
metrics


{'eval_loss': 2.9896292686462402,
 'eval_runtime': 2.6889,
 'eval_samples_per_second': 56.901,
 'eval_steps_per_second': 14.504,
 'epoch': 6.0}

In [22]:
trainer.save_model("./final_engl_chatbot")
tokenizer.save_pretrained("./final_engl_chatbot")

print("Model saved successfully!")


Model saved successfully!


In [23]:
def en_chatbot(text):
    # Prepare inputs with max_length (REMOVE WARNINGS)
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128  # safe for mT5
    ).to(model.device)

    # Generate with repetition control
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        min_length=5,
        num_beams=4,
        no_repeat_ngram_size=3,        # IMPORTANT: reduces repetition
        repetition_penalty=2.0,        # stronger penalty
        length_penalty=1.0,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [30]:
print("ENGLISH Chatbot Ready! Type 'exit' to stop.\n")

while True:
    user_msg = input("YOU: ")

    if user_msg.lower() in ["exit", "quit", "stop"]:
        print("Chatbot session ended.")
        break

    bot_reply = en_chatbot(user_msg)
    print("chat bot:", bot_reply)


ENGLISH Chatbot Ready! Type 'exit' to stop.

YOU: my friend is not talking,its stressing me a bit
chat bot: It sounds like you're not talking to your friend about your feelings. I would suggest that you have a conversation with your friend, but it could be helpful to talk to the friend about how you feel and what you are experiencing.
YOU: quit
Chatbot session ended.
